In [ ]:
import drawSvg as draw
from drawSvg.widgets import DrawingWidget
import hyperbolic.poincare.shapes as hyper  # pip3 install hyperbolic

def from_list_to_matrix(ls):
    global cout
    matrix = [[0 for i in range(cout)] for j in range(cout)]
    for key, value in ls.items():
        for k in value:
            matrix[key][k] = 1
            matrix[k][key] = 1
    return matrix

# Create drawing
d = draw.Drawing(2, 2, origin='center')
d.setRenderSize(500)
d.append(draw.Circle(0, 0, 1, fill='#ffd83c'))
group = draw.Group()
d.append(group)

cout = 0
matrix = {}
pps = {}
# Update the drawing based on user input
click_list = []
def redraw(points):
    group.children.clear()
    for x1, y1 in points:
        for x2, y2 in points:
            if (x1, y1) == (x2, y2): continue
            p1 = hyper.Point.fromEuclid(x1, y1)
            p2 = hyper.Point.fromEuclid(x2, y2)
            if p1.distanceTo(p2) <= 2:
                line = hyper.Line.fromPoints(*p1, *p2, segment=True)
                group.draw(line, hwidth=0.2, fill='white')
    for x, y in points:
        p = hyper.Point.fromEuclid(x, y)
        group.draw(hyper.Circle.fromCenterRadius(p, 0.1),
                   fill='green')
        if (x,y) in pps:
            group.draw(draw.Text(str(pps[(x,y)]), 100,path=p, fill='black'))
redraw(click_list)

# Create interactive widget and register mouse events
widget = DrawingWidget(d)
@widget.mousedown
def mousedown(widget, x, y, info):
    global cout
    pps[(x,y)] = cout
    cout += 1
    if (x**2 + y**2) ** 0.5 + 1e-5 < 1:
        for x1,y1 in click_list:
            p1 = hyper.Point.fromEuclid(x, y)
            p2 = hyper.Point.fromEuclid(x1, y1)
            if p1.distanceTo(p2) <= 2:
                if pps[(x,y)] in matrix:
                    matrix[pps[(x,y)]].add(pps[(x1,y1)])
                else:
                    matrix[pps[(x,y)]] = set([pps[(x1,y1)]])
        click_list.append((x, y))
        
    print('-'*(2*cout-1))
    for line in from_list_to_matrix(matrix):
        for el in line:
            print(el, end=' ')
        print()
    print('-'*(2*cout-1))
    redraw(click_list)
    widget.refresh()
@widget.mousemove
def mousemove(widget, x, y, info):
    if (x**2 + y**2) ** 0.5 + 1e-5 < 1:
        redraw(click_list + [(x, y)])
    widget.refresh()
    
widget